### Imports

In [5]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
import pyEX as p

### Create Client

In [6]:
c = p.Client(api_token='Your Token', version='stable')

Reference: https://iexcloud.io/blog/how-to-get-market-data-in-python

### Try to obtain apple dict

In [7]:
apple = c.quote(symbol='AAPL')
#apple

### Create a dataframe for constructing SP500 portfolio

In [8]:
col = ['Ticker', 'Px', 'Market Cap', 'Number of shares to buy']
df = pd.DataFrame(columns = col)
df

,Ticker,Px,Market Cap,Number of shares to buy


### Load the file for ticker

In [9]:
stocks = pd.read_csv('sp_500_stocks.csv')

### Batch API Call

Split the list of stocks into groups of 100 and make a batch API call for each group

In [10]:
API_TOKEN = 'Your Token' # Should not be shared

In [11]:
def chunks(lst, n):
    """Yield successive n-sized chunk from lst"""
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

Reference: https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks

In [12]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])
for sym_string in symbol_strings:
    #print(sym_string)
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=quote&symbols={sym_string}&token={API_TOKEN}'
    #print(batch_api_call_url)
    sp500_Data = requests.get(batch_api_call_url).json()
    for sym in sym_string.split(','):
        df = df.append(
                     pd.Series([sym, sp500_Data[sym]['quote']['latestPrice'], sp500_Data[sym]['quote']['marketCap'], 'N/A'], 
                               index = col), 
                               ignore_index = True)
df

,Ticker,Px,Market Cap,Number of shares to buy
0,A,129.32,38843345345,N/A
1,AAL,13.51,8770153181,N/A
2,AAP,200.58,12254952396,N/A
3,AAPL,157.44,2569332791040,N/A
4,ABBV,147.15,260272125937,N/A
...,...,...,...,...
500,ZBRA,383.92,20378373781,N/A
501,ZION,62.45,9465816596,N/A
502,ZTS,181.39,85610743506,N/A
503,ZION,62.45,9465816596,N/A


### Calculate the number of shares to buy

In [45]:
pSize = 10000000

In [46]:
position_size = val/len(df.index)
for i in range(0,len(df.index)):
    df.loc[i, 'Number of shares to buy'] = math.floor(position_size/df.loc[i, 'Px'])

df

,Ticker,Px,Market Cap,Number of shares to buy
0,A,129.32,38843345345,153
1,AAL,13.51,8770153181,1465
2,AAP,200.58,12254952396,98
3,AAPL,157.44,2569332791040,125
4,ABBV,147.15,260272125937,134
...,...,...,...,...
500,ZBRA,383.92,20378373781,51
501,ZION,62.45,9465816596,317
502,ZTS,181.39,85610743506,109
503,ZION,62.45,9465816596,317


### Output in a Excel file

In [52]:
writer = pd.ExcelWriter('equalWeight_SP500_ETF.xlsx', engine = 'xlsxwriter')
df.to_excel(writer, 'equalWeight_SP500_ETF', index = False)

In [53]:
background_Color = '#1a3432'
font_Color = '#ffffff'
string_format = writer.book.add_format(
    {
        'font_color': font_Color,
        'bg_color': background_Color,
        'border': 1    
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_Color,
        'bg_color': background_Color,
        'border': 1    
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_Color,
        'bg_color': background_Color,
        'border': 1    
    }
)

In [54]:
writer.sheets['equalWeight_SP500_ETF'].set_column('A:A', 18, string_format)
writer.sheets['equalWeight_SP500_ETF'].set_column('B:B', 18, string_format)
writer.sheets['equalWeight_SP500_ETF'].set_column('C:C', 18, string_format)
writer.sheets['equalWeight_SP500_ETF'].set_column('D:D', 18, string_format)
writer.save()

In [55]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Px', dollar_format],
    'C': ['Market Cap', dollar_format],
    'D': ['Number of shares to buy', integer_format]
}

In [57]:
for col in column_formats.keys():
    writer.sheets['equalWeight_SP500_ETF'].set_column(f'{col}:{col}', 18, column_formats[col][1])
writer.save()

#### Functions related to indices are not available on IEX cloud. Hence, we can only use Excel to load the tickers for now.